<a href="https://colab.research.google.com/github/mohammadreza76/cifar10-with-deeplearning-architecture/blob/master/cifar10_deepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries and dataset

In [1]:
from keras.datasets import cifar10
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from keras.utils import np_utils
from keras.models import Sequential
from matplotlib import pyplot
from keras.regularizers import l2
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
import tensorflow as tf

## Load data

In [2]:
(X_train,y_train),(X_test,y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 3s 0us/step


In [ ]:
X_train.shape

(50000, 32, 32, 3)

## preparing data

In [ ]:


X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train_flatten = X_train.reshape(X_train.shape[0],-1)
X_test_flatten = X_test.reshape(X_test.shape[0],-1)

# Standardize data to have feature values between 0 and 1.
X_train = X_train_flatten/255.0
X_test = X_test_flatten/255.0

In [ ]:
X_test.shape

(10000, 3072)

In [3]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test) 
number_classes = y_test.shape[1]
number_classes

10

In [ ]:
X_train.shape

(50000, 3072)

In [ ]:
y_test.shape

(10000, 10)

In [ ]:
y_train = y_train

In [ ]:
y_train.shape

(50000, 10)

## Define the baseline model

In [ ]:
def model():
  #create model
  model = Sequential()
  model.add(Dense(1060,activation='relu'))
  model.add(Dense(900,activation='relu'))
  model.add(Dense(830,activation='relu'))
  model.add(Dense(780,activation='relu'))
  model.add(Dense(780,activation='relu'))
  model.add(Dense(512,activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(512,activation='relu'))
  model.add(Dense(512,activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(512,activation='relu'))
  model.add(Dense(256,activation='relu'))
  model.add(Dense(128,activation='relu'))
  model.add(Dense(number_classes,activation='softmax',kernel_initializer='normal'))

  #compile model
  model.compile(loss= 'categorical_crossentropy',optimizer = 'adam', metrics=['accuracy'])
  return model

In [ ]:
#build model
model = model()

# Fit the model
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs= 100,batch_size= 128,shuffle=True,verbose=0)

# evaluation of the model
scores = model.evaluate(X_test,y_test,verbose=0)
print("model Error: %.2f%%" % (100-scores[1]*100))

model Error: 54.39%


In [ ]:
scores = model.evaluate(X_train,y_train,verbose=0)
print("model Error: %.2f%%" % (100-scores[1]*100))

model Error: 7.96%


## Define the CNN model

In [ ]:
def cnn_model():
  #create model
  #conv -> maxpool -> conv -> conv -> maxpool -> conv -> maxpool -> conv -> conv -> maxpool -> dense -> dense -> softmax
  cnn_model = Sequential()
  cnn_model.add(Conv2D(filters=32,kernel_size=(3,3),strides= (1,1),padding='same',activation='relu',kernel_initializer='he_uniform'))
  cnn_model.add(MaxPooling2D(pool_size=(2,2),strides=1,padding='valid'))
  cnn_model.add(Conv2D(filters=128,kernel_size=(3,3),strides= (2,2),padding='same',activation='relu',kernel_initializer='he_uniform',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
  cnn_model.add(Conv2D(filters=64,kernel_size=(5,5),strides= (1,1),padding='valid',activation='relu',kernel_initializer='he_uniform'))
  cnn_model.add(MaxPooling2D(pool_size=(2,2),strides=1,padding='valid'))
  cnn_model.add(Dropout(0.2))
  cnn_model.add(Conv2D(filters=64,kernel_size=(5,5),strides= (1,1),padding='same',activation='relu',kernel_initializer='he_uniform'))
  cnn_model.add(MaxPooling2D(pool_size=(4,4),strides=1,padding='valid'))
  cnn_model.add(Conv2D(filters=64,kernel_size=(3,3),strides= (1,1),padding='same',activation='relu',kernel_initializer='he_uniform'))
  cnn_model.add(Conv2D(filters=64,kernel_size=(5,5),strides= (2,2),padding='same',activation='relu',kernel_initializer='he_uniform'))
  cnn_model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))
  cnn_model.add(Dropout(0.25))
  cnn_model.add(Flatten())
  cnn_model.add(Dense(256,activation='relu',kernel_initializer='GlorotNormal',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
  cnn_model.add(Dense(128,activation='relu',kernel_initializer='GlorotNormal'))
  cnn_model.add(Dense(number_classes,activation='softmax'))

  #compile model
  cnn_model.compile(loss= 'categorical_crossentropy',optimizer = 'adam', metrics=['accuracy'])

  return cnn_model

In [4]:
#prepare data (convert type and normalized)
X_train_cnn = X_train.astype('float32')
X_test_cnn = X_test.astype('float32')


# Standardize data to have feature values between 0 and 1.
X_train_new = X_train_cnn/255.0
X_test_new = X_test_cnn/255.0

In [ ]:
X_train_new.shape

(50000, 32, 32, 3)

In [ ]:
X_test_cnn.shape

(10000, 32, 32, 3)

In [ ]:
y_train.shape

(50000, 1)

In [ ]:
y_test.shape

(10000, 1)

In [ ]:
#build model
CNN_model = cnn_model()

# Fit the model
CNN_model.fit(X_train_new,y_train,validation_data=(X_test_new,y_test),epochs= 100,batch_size= 128,shuffle=True,verbose=0)

# evaluation of the model
scores = CNN_model.evaluate(X_test_new,y_test,verbose=0)
print("model Error: %.2f%%" % (100-scores[1]*100))

model Error: 22.35%


In [ ]:
print(scores[1]*100)

77.64999866485596


In [ ]:
scores_new = CNN_model.evaluate(X_train_new,y_train,verbose=0)
print("model Error: %.2f%%" % (100-scores_new[1]*100))

model Error: 3.97%


In [ ]:
print(CNN_model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16, 16, 128)       36992     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 12, 12, 64)        204864    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 11, 11, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 11, 11, 64)       

## Transfer learning

Pre-Trained Model as Feature Extractor in Model

In [5]:
# load VGG model
vgg_model_1 = VGG16(input_shape=(32,32,3),include_top= False)

58892288/58889256 [==============================] - 1s 0us/step


In [6]:
#reshape one of the data for VGG model
X_vgg16_1 = X_train_new[0].reshape((1,X_train_new[0].shape[0], X_train_new[0].shape[1], X_train_new[0].shape[2] ))
X_vgg16_1.shape

(1, 32, 32, 3)

In [8]:
# add new classifier layers
 
flatten_layer = Flatten()(vgg_model_1.layers[-1].output)
full_layer_1 = Dense(2048,activation='relu')(flatten_layer)
full_layer_2 = Dense(1600,activation='relu')(full_layer_1)
output = Dense(10,activation='softmax')(full_layer_2)

#define new model
vgg_model_1_new = Model(inputs = vgg_model_1.inputs, outputs = output)

#model summary
vgg_model_1_new.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)        

In [12]:
features = vgg_model_1_new.predict(X_vgg16_1)
print(features.shape)
print(features)

(1, 10)
[[0.0767149  0.07703622 0.18634455 0.08396413 0.09703261 0.13871315
  0.0865022  0.07681928 0.07579745 0.10107552]]
